In [3]:
import pandas as pd
import numpy as np

from google.colab import drive

# Bibliotecas de pré-processamento
from sklearn.preprocessing import LabelEncoder

# Bibliotecas de plotting
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.datasets import make_classification
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier
from sklearn.linear_model import RidgeClassifierCV, LogisticRegression, PassiveAggressiveClassifier, SGDClassifier
from sklearn.svm import LinearSVC, SVC, NuSVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.semi_supervised import LabelPropagation, LabelSpreading
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.dummy import DummyClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import numpy as np
from sklearn.linear_model import Perceptron
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

In [ ]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Data/merged_df.csv')

In [ ]:
from sklearn.preprocessing import StandardScaler

df = df.drop(['record_id', 'idade_faixa'], axis=1)

X_afetam_eficacia = df[['idade_diagnostico','primeiro_IMC','ultima_informacao_paciente', 'tempo_seguimento', 'recidiva_distancia', 'recidiva_regional', 'recidiva_local']]
X_afetam_eficacia[['tempo_seguimento']] = StandardScaler().fit_transform(X_afetam_eficacia[['tempo_seguimento']])

In [ ]:
km = KMeans( n_clusters = 2, init = 'random', max_iter = 300, n_init = 100, random_state = 52 )
km.fit( X_afetam_eficacia )
y_km = km.predict( X_afetam_eficacia )
df['eficacia_tratamento'] = y_km

In [ ]:
print(len(df.query("idade_diagnostico > 80 and ultima_informacao_paciente == 3 and eficacia_tratamento == 0")))
print(len(df.query("idade_diagnostico > 80 and ultima_informacao_paciente == 3 and eficacia_tratamento == 1")))

# Este é um sistema de classificação de tratamento em que "0" significa que o tratamento foi bom e "1" significa 
# que foi ruim. Há dois casos apresentados: no primeiro, uma pessoa idosa de 80 anos sobreviveu ao tratamento, indicando 
# que ele foi bom. No segundo caso, não houve sobreviventes entre as pessoas mais velhas e, portanto, o tratamento foi classificado 
# como negativo para essa faixa etária. É importante lembrar que a eficácia do tratamento pode variar de acordo com vários fatores e 
# que cada caso é único.

0
38


In [ ]:
df.query("idade_diagnostico < 30 and ultima_informacao_paciente == 0 and eficacia_tratamento == 1")

print(len(df.query("idade_diagnostico < 30 and ultima_informacao_paciente == 0 and eficacia_tratamento == 0")))
print(len(df.query("idade_diagnostico < 30 and ultima_informacao_paciente == 0 and eficacia_tratamento == 1")))

# Este modelo classifica o tratamento como bom (0) ou ruim (1). Ele foi testado em dois casos de mulheres com menos
# de 30 anos: uma morreu e a outra sobreviveu. O modelo foi capaz de classificar corretamente cada caso, indicando que 
# tem uma boa capacidade de classificação.

12
0


In [ ]:
df = df.query("tratamento == 0 or tratamento == 1")[:]

In [ ]:
import numpy as np

df = df.astype('float64')
df = df[np.isfinite(df).all(1)]

In [ ]:
# Divisao do conjunto Teste/Treino
from sklearn.model_selection import train_test_split

X = df.drop(['tratamento'], axis=1)
y = df['tratamento']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 73)

In [ ]:
!pip install lazypredict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from lazypredict.Supervised import LazyClassifier

clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)

100%|██████████| 29/29 [00:34<00:00,  1.20s/it]


In [ ]:
# Metricas antes do ajuste dos hiperparametros
models

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
AdaBoostClassifier,0.67,0.67,0.67,0.67,0.47
RandomForestClassifier,0.67,0.67,0.67,0.67,1.69
LGBMClassifier,0.67,0.66,0.66,0.66,8.76
ExtraTreesClassifier,0.66,0.66,0.66,0.66,0.59
LinearSVC,0.66,0.66,0.66,0.66,1.36
RidgeClassifierCV,0.66,0.66,0.66,0.66,0.15
LogisticRegression,0.66,0.66,0.66,0.66,0.31
SVC,0.66,0.66,0.66,0.66,0.95
LinearDiscriminantAnalysis,0.66,0.65,0.65,0.66,0.07


In [ ]:
classifiers = [AdaBoostClassifier(), 
               RandomForestClassifier(),
               ExtraTreesClassifier(),
               RidgeClassifierCV(), 
               LinearSVC(),
               LogisticRegression(), 
               SVC(), 
               LGBMClassifier(),
               LinearDiscriminantAnalysis(), 
               RidgeClassifier(), 
               CalibratedClassifierCV(), 
               XGBClassifier(),
               NearestCentroid(), 
               SGDClassifier(), 
               BaggingClassifier(),
               BernoulliNB(),
               PassiveAggressiveClassifier(), 
               Perceptron(),
               NuSVC(), 
               DecisionTreeClassifier(), 
               KNeighborsClassifier(),
               LabelSpreading(),
               LabelPropagation(), 
               ExtraTreeClassifier(),
               GaussianNB(),
               DummyClassifier(),
               QuadraticDiscriminantAnalysis()]

precisions = []
recalls = []

for classifier in classifiers:
    clf = classifier.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    precisions.append(precision_score(y_test, y_pred))
    recalls.append(recall_score(y_test, y_pred))

In [ ]:
models['Precision'] = 0
models['Recall'] = 0   

for i in range(0, len(precisions)):
  models['Precision'].iloc[i] = precisions[i]
  models['Recall'].iloc[i] = recalls[i]

models

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken,Precision,Recall
Model,,,,,,,
AdaBoostClassifier,0.67,0.67,0.67,0.67,0.47,0.66,0.62
RandomForestClassifier,0.67,0.67,0.67,0.67,1.69,0.65,0.62
LGBMClassifier,0.67,0.66,0.66,0.66,8.76,0.64,0.61
ExtraTreesClassifier,0.66,0.66,0.66,0.66,0.59,0.65,0.60
LinearSVC,0.66,0.66,0.66,0.66,1.36,0.00,0.00
RidgeClassifierCV,0.66,0.66,0.66,0.66,0.15,0.63,0.60
LogisticRegression,0.66,0.66,0.66,0.66,0.31,0.00,0.00
SVC,0.66,0.66,0.66,0.66,0.95,0.65,0.60
LinearDiscriminantAnalysis,0.66,0.65,0.65,0.66,0.07,0.64,0.59


# Accuracy: 0.6641550053821313
# Balanced Accuracy: 0.6591726283627692
# ROC AUC: 0.6591726283627691
# F1 Score: 0.6622626319221622

In [ ]:
clf = AdaBoostClassifier()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.6716899892357373

##GridSearch

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

# Grid Search do modelo AdaBoostClasifier

model = AdaBoostClassifier()

# Escolha dos hiperparametros
params = {
          'n_estimators': [50, 100, 150],
          'learning_rate': [0.01, 0.1],
          'algorithm': ['SAMME', 'SAMME.R'],
           'random_state': [73]
          }

grid_searchAda = GridSearchCV(estimator=model, param_grid=params, cv=5)

grid_searchAda.fit(X_train, y_train)

print(grid_searchAda.best_params_)

# {'algorithm': 'SAMME.R', 'learning_rate': 0.1, 'n_estimators': 150, 'random_state': 73}

{'algorithm': 'SAMME.R', 'learning_rate': 0.1, 'n_estimators': 150, 'random_state': 73}
0.6837443194516875
0.673842841765339


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, roc_auc_score, f1_score

y_pred = grid_searchAda.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred, multi_class='ovr')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Balanced Accuracy:", balanced_accuracy)
print("ROC AUC:", roc_auc)
print("F1 Score:", f1)

# Accuracy: 0.673842841765339
# Balanced Accuracy: 0.6680755831284
# ROC AUC: 0.6680755831284001
# F1 Score: 0.6713062718325303

Accuracy: 0.673842841765339
Balanced Accuracy: 0.6680755831284
ROC AUC: 0.6680755831284001
F1 Score: 0.6713062718325303


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Grid Search do modelo RandomForestClasifier

model = RandomForestClassifier()

# Escolha dos hiperparametros
params = {
          'n_estimators': [50, 100, 150],
          'max_depth': [5,10,15],
          'max_features':['sqrt','log2', None],
          'random_state': [73]
          }

grid_searchRF = GridSearchCV(estimator=model, param_grid=params, cv=5)

grid_searchRF.fit(X_train, y_train)

print(grid_searchRF.best_params_)

# {'max_depth': 10, 'max_features': 'sqrt', 'n_estimators': 100, 'random_state': 73}

{'max_depth': 10, 'max_features': 'sqrt', 'n_estimators': 100, 'random_state': 73}
0.6911463266674472
0.6598493003229279


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, roc_auc_score, f1_score

y_pred = grid_searchRF.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred, multi_class='ovr')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Balanced Accuracy:", balanced_accuracy)
print("ROC AUC:", roc_auc)
print("F1 Score:", f1)

# Accuracy: 0.6598493003229279
# Balanced Accuracy: 0.6543916275430359
# ROC AUC: 0.6543916275430359
# F1 Score: 0.6575477826730564

Accuracy: 0.6598493003229279
Balanced Accuracy: 0.6543916275430359
ROC AUC: 0.6543916275430359
F1 Score: 0.6575477826730564


In [ ]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV

model = LGBMClassifier()

# Escolha dos parametros
params = {
    'max_depth': [5, 10],
    'learning_rate': [0.01, 0.05],
    'n_estimators': [50, 100],
    'random_state': [73]
}

grid_searchLGBM = GridSearchCV(estimator=model, param_grid=params, cv=5)

grid_searchLGBM.fit(X_train, y_train)

print(grid_searchLGBM.best_params_)

# {'learning_rate': 0.05, 'max_depth': 10, 'n_estimators': 50, 'random_state': 73}

{'learning_rate': 0.05, 'max_depth': 10, 'n_estimators': 50, 'random_state': 73}
0.6842125988442014
0.6727664155005382


In [ ]:
# Grid Search do modelo LGBM


from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, roc_auc_score, f1_score

y_pred = grid_searchLGBM.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred, multi_class='ovr')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Balanced Accuracy:", balanced_accuracy)
print("ROC AUC:", roc_auc)
print("F1 Score:", f1)

# Accuracy: 0.6727664155005382
# Balanced Accuracy: 0.667523660481407
# ROC AUC: 0.667523660481407
# F1 Score: 0.6706789482402674

Accuracy: 0.6727664155005382
Balanced Accuracy: 0.667523660481407
ROC AUC: 0.667523660481407
F1 Score: 0.6706789482402674


##RandomizedSearchCV

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

model = AdaBoostClassifier()

# Escolha dos parametros
params = {
          'n_estimators': [50, 100, 150],
          'learning_rate': [0.01, 0.1],
          'algorithm': ['SAMME', 'SAMME.R'],
           'random_state': [73]
          }

randomizedSearchAda = RandomizedSearchCV(estimator=model, param_distributions=params, cv=5, n_iter=50)

randomizedSearchAda.fit(X_train, y_train)

print(randomizedSearchAda.best_params_)
# {'random_state': 73, 'n_estimators': 150, 'learning_rate': 0.1, 'algorithm': 'SAMME.R'}

{'random_state': 73, 'n_estimators': 150, 'learning_rate': 0.1, 'algorithm': 'SAMME.R'}
0.6837443194516875
0.673842841765339


In [ ]:
# Random Search do modelo AdaBoostClasifier

from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, roc_auc_score, f1_score

y_pred = randomizedSearchAda.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred, multi_class='ovr')
f1 = f1_score(y_test, y_pred, average='weighted')

# imprimindo as métricas
print("Accuracy:", accuracy)
print("Balanced Accuracy:", balanced_accuracy)
print("ROC AUC:", roc_auc)
print("F1 Score:", f1)

# Accuracy: 0.673842841765339
# Balanced Accuracy: 0.6680755831284
# ROC AUC: 0.6680755831284001
# F1 Score: 0.6713062718325303

Accuracy: 0.673842841765339
Balanced Accuracy: 0.6680755831284
ROC AUC: 0.6680755831284001
F1 Score: 0.6713062718325303


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()

# Escolha dos hiperparametros
random_grid = {
          'n_estimators': [50, 100, 150],
          'max_depth': [5,10,15],
          'max_features':['sqrt','log2', None],
          'random_state': [73]
               }

randomizedSearchRF = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

randomizedSearchRF.fit(X_train, y_train)

print(randomizedSearchRF.best_params_)

# {'random_state': 73, 'n_estimators': 150, 'max_features': 'sqrt', 'max_depth': 10}

Fitting 3 folds for each of 27 candidates, totalling 81 fits
{'random_state': 73, 'n_estimators': 150, 'max_features': 'sqrt', 'max_depth': 10}
0.6809787626962143
0.6641550053821313


In [ ]:
# Random Search do modelo RandomForestClasifier

from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, roc_auc_score, f1_score

y_pred = randomizedSearchRF.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred, multi_class='ovr')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Balanced Accuracy:", balanced_accuracy)
print("ROC AUC:", roc_auc)
print("F1 Score:", f1)

# Accuracy: 0.6641550053821313
# Balanced Accuracy: 0.6591726283627692
# ROC AUC: 0.6591726283627691
# F1 Score: 0.6622626319221622

Accuracy: 0.6641550053821313
Balanced Accuracy: 0.6591726283627692
ROC AUC: 0.6591726283627691
F1 Score: 0.6622626319221622


In [ ]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV

model = LGBMClassifier()

# Escolha dos hiperparametros
params = {
    'max_depth': [5, 10],
    'learning_rate': [0.01, 0.05],
    'n_estimators': [50, 100],
    'random_state': [73]
}

randomizedSearchLGBM = RandomizedSearchCV(estimator=model, param_distributions=params, cv=5)

randomizedSearchLGBM.fit(X_train, y_train)

print(randomizedSearchLGBM.best_params_)
print(randomizedSearchLGBM.best_score_)

{'random_state': 73, 'n_estimators': 50, 'max_depth': 10, 'learning_rate': 0.05}
0.6842125988442014
0.6727664155005382


In [ ]:
# Random Search do modelo RandomForestClasifier

from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, roc_auc_score, f1_score

y_pred = randomizedSearchLGBM.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred, multi_class='ovr')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Balanced Accuracy:", balanced_accuracy)
print("ROC AUC:", roc_auc)
print("F1 Score:", f1)

# Accuracy: 0.6727664155005382
# Balanced Accuracy: 0.667523660481407
# ROC AUC: 0.667523660481407
# F1 Score: 0.6706789482402674

Accuracy: 0.6727664155005382
Balanced Accuracy: 0.667523660481407
ROC AUC: 0.667523660481407
F1 Score: 0.6706789482402674


In [74]:
data_random = {'modelos': ['AdaBoost', 'RandomForest', 'LGBM'],
               'valores': [0.683, 0.680, 0.684],
               'valores_antigos': [0.673, 0.664, 0.672]}

df = pd.DataFrame(data_random)

df = df.rename(columns={'valores_antigos': 'Acurácia antiga', 'valores': 'Acurácia Nova'})

df['tamanho'] = df['Acurácia Nova']

fig = px.bar(df, x='modelos', y=['Acurácia antiga', 'Acurácia Nova'],
             labels={'Acurácia antiga': 'Acurácia antiga', 'Acurácia Nova': 'Acurácia Nova'},
             color_discrete_sequence=px.colors.qualitative.Pastel,
             title='Acurácia Sem Random Search / Acurácia Com Random Search',
             barmode='group')

fig.update_traces(texttemplate='%{y:.3f}', textposition='outside')
fig.show()

In [78]:
data = {'modelos': ['AdaBoost', 'RandomForest', 'LGBM'], 
        'valores': [0.683, 0.691, 0.684],
        'valores_antigos': [0.673, 0.659, 0.672]}

df = pd.DataFrame(data)

df = df.rename(columns={'valores_antigos': 'Acurácia antiga', 'valores': 'Acurácia Nova'})

fig = px.bar(df, x='modelos', y=['Acurácia antiga', 'Acurácia Nova'],
             labels={'Acurácia antiga': 'Acurácia antiga', 'Acurácia Nova': 'Acurácia Nova'},
             color_discrete_sequence=px.colors.qualitative.Pastel,
             title='Acurácia Sem Grid Search / Acurácia Com Grid Search',
             barmode='group')

fig.update_traces(texttemplate='%{y:.3f}', textposition='outside')

fig.show()

##Conclusão

Logo apartir destas métricas o melhor modelo é o RandomForestClassifier com 69.1% de acurácia utilizando estes parametros: 
{'n_estimators': [50, 100, 150], 'max_depth': [5,10,15], 'max_features':['sqrt','log2', None], 'random_state': [73]}
